In [1]:
#import sys
from osgeo import gdal
import fiona
import pandas as pd

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
data_path = '../../../../Praktikum_data/geodata_vienna/'

In [3]:
sectors = pd.read_json(data_path + r'vienna_sectors.json', orient='index')
trees = pd.read_json(data_path + r'vienna_trees.json', orient='index')

In [4]:
sectors

,id,sector,coor_min_x,coor_max_y,coor_max_x,coor_min_y
0,0,12/2,16.163534,48.333222,16.197313,48.310694
1,1,13/1,16.197252,48.333258,16.231016,48.310740
2,2,13/2,16.230970,48.333284,16.264719,48.310775
3,3,14/1,16.264688,48.333300,16.298423,48.310801
4,4,14/2,16.298406,48.333306,16.332126,48.310817
...,...,...,...,...,...,...
145,145,57/4,16.499997,48.130843,16.533653,48.108307
146,146,58/3,16.533567,48.130790,16.567239,48.108244
147,147,58/4,16.567138,48.130727,16.600824,48.108172
148,148,59/3,16.600708,48.130655,16.634409,48.108089


In [5]:
# TODO: loop over all sectors if needed
sector = '36/1'
sector_to_split = sectors[sectors['sector'] == sector]
sector_to_split

,id,sector,coor_min_x,coor_max_y,coor_max_x,coor_min_y
67,67,36/1,16.399422,48.243358,16.433109,48.220851


## Split the chosen raster

In [6]:
height = float(sector_to_split['coor_max_y']) - float(sector_to_split['coor_min_y'])
width = float(sector_to_split['coor_max_x']) - float(sector_to_split['coor_min_x'])
print('width: {}, height: {}'.format(width, height))

width: 0.0336868222999982, height: 0.022506384799996226


In [7]:
# TODO: loop over all sector splits if needed
# Example for 0.1 width, 0.1 height

#1. split von 36/1
#output_coor_min_x = float(sector_to_split['coor_min_x'])
#output_coor_max_y = float(sector_to_split['coor_max_y'])
#output_coor_max_x = float(sector_to_split['coor_min_x']) + 0.1 * width
#output_coor_min_y = float(sector_to_split['coor_max_y']) - 0.1 * height

# 80. split von 36/1
output_coor_min_x = float(sector_to_split['coor_max_x']) - 0.1 * width
output_coor_max_y = float(sector_to_split['coor_min_y']) + 0.3 * height
output_coor_max_x = float(sector_to_split['coor_max_x']) 
output_coor_min_y = float(sector_to_split['coor_min_y']) + 0.2 * height

output_height = output_coor_max_y - output_coor_min_y
output_width = output_coor_max_x - output_coor_min_x
print(output_coor_min_x, output_coor_max_y, output_coor_max_x, output_coor_min_y)
print(output_height, output_width)

16.429740302669998 48.22760328294 16.4331089849 48.22535264446
0.0022506384799996226 0.0033686822300005304


In [8]:
# TODO: continue in a loop over all sector splits if needed
# Only for clipping images
from gdalconst import GA_ReadOnly

data = gdal.Open(data_path + 'ORTHO/36_1_op_2019.jpg', GA_ReadOnly)
geoTransform = data.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * data.RasterXSize
miny = maxy + geoTransform[5] * data.RasterYSize
print(minx,maxy,maxx,miny)

#1. split von 36/1
#new_minx = minx
#new_maxy = maxy
#new_maxx = 5250.0
#new_miny = 344750.0

# 80. split von 36/1
new_minx = maxx - 250.0
new_maxy = miny + 750.0
new_maxx = maxx
new_miny = miny + 500.0

#1. split von 36/1
#ds = gdal.Translate(data_path + '/ORTHO/clipped/output.jpg', data, projWin = [5000.0, 345000.0, 5250.0, 344750.0])

# 80. split von 36/1
ds = gdal.Translate(data_path + 'ORTHO/clipped/output2.jpg', data, projWin = [new_minx,new_maxy,new_maxx,new_miny])
ds = None

5000.0 345000.0 7500.000000000004 342500.0


In [9]:
tree_indizes = []

output_trees = trees[(trees['coor_x'] > output_coor_min_x) & (trees['coor_x'] < output_coor_max_x) 
      & (trees['coor_y'] > output_coor_min_y) & (trees['coor_y'] < output_coor_max_y)]

print(len(trees))
print(len(output_trees))

202137
363


In [10]:
output_trees

,id,tree_id,type,height,coor_x,coor_y
10513,10513,61634,Populus x canescens (Graupappel),26-30 m,16.432603,48.227236
10532,10532,61789,Quercus robur (Stieleiche),6-10 m,16.432576,48.225812
10977,10977,50781,Betula pendula (Weißbirke),11-15 m,16.432116,48.227523
11215,11215,62355,Betula pendula (Weißbirke),11-15 m,16.431983,48.227074
11243,11243,62313,Populus x canescens (Graupappel),21-25 m,16.431912,48.226609
...,...,...,...,...,...,...
115699,115699,50776,Populus nigra (Schwarzpappel),26-30 m,16.431998,48.227521
200246,200246,62185,Populus nigra (Schwarzpappel),26-30 m,16.429803,48.226797
201867,201867,61678,Populus x canescens (Graupappel),21-25 m,16.432085,48.225988
201930,201930,317769,Robinia pseudoacacia (Scheinakazie),11-15 m,16.430047,48.226679


In [11]:
#print(output_coor_min_x, output_coor_max_y, output_coor_max_x, output_coor_min_y)
#print(output_height, output_width)
string = ''
#text_file = open(data_path + 'ORTHO/clipped/output.txt', 'w')
text_file = open(data_path + 'ORTHO/clipped/output3.txt', 'w')
# TODO: recalculate size, also using with metadeta for each tree
size = float(75/1667)
for tree in range(len(output_trees)):
    x = (output_trees.iloc[tree, 4] - output_coor_min_x) / output_width
    y = (output_coor_max_y - output_trees.iloc[tree, 5]) / output_height
    string += '{} {} {} {} {}\n'.format(0, x, y, size, size)

#print(string)
# write the string in file without the last new line
text_file.write(string[:-1])
# close the file
text_file.close() 